# Union de las tablas

El objetivo de este notebook es unir las tres tablas para obtener un dataset manipulable y posteriormente sacar insights para negocio.

## 1. Librerias

In [ ]:
import pickle
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Dataset

In [ ]:
df_calendar = pd.read_csv('daily_calendar_with_events.csv')
df_prices = pd.read_csv('item_prices.csv')
df_sales = pd.read_csv('item_sales.csv')

### Optimización de df_sales

In [ ]:
df_sales.head()

In [ ]:
# transformamos las numéricas de sales para optimizar la memoria a uint8, ya que son dias bool
for i in df_sales:
    list_1 = ["id","item","category","department", "store", "store_code", "region"]
    if i in(list_1):
        continue
    else:
        df_sales[i] = df_sales[i].astype("uint8")

La función melt permite deshacer la estructura de las columnas y apilarlas en una sola columna, mientras se conserva la información de las variables identificadoras.

En nuestro caso, al aplicar la función melt al DataFrame sales, especificamos las variables identificadoras en el parámetro id_vars, el nombre de la columna que almacenará las variables de fecha en el parámetro var_name, y el nombre de la columna que almacenará los valores de las unidades en el parámetro value_name.

In [ ]:
df_sales_net = pd.melt(df_sales,
                       id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'],  # columnas que no sufriran cambios
                       var_name='d',      # nombre de la nueva columna que crearé con las columnas d_
                       value_name='quantity')   # añade los valores de las columnas d_

In [ ]:
df_sales_net.head()

### Optimización de df_calendar

In [ ]:
df_calendar.head(10)

In [ ]:
df_calendar['weekday_int'].unique()

In [ ]:
#transformamos calendar para optimizar la memoria
df_calendar["weekday_int"] = df_calendar["weekday_int"].astype("uint8")

In [ ]:
# Convertir la columna 'date' a datetime
df_calendar['date'] = pd.to_datetime(df_calendar['date'])
df_calendar["week"] = df_calendar["date"].dt.isocalendar().week.astype(str)
df_calendar["year"] = df_calendar["date"].dt.isocalendar().year.astype(str)
df_calendar["year_week"] = df_calendar["year"] + "_" + df_calendar["week"]

df_calendar.head()

In [ ]:
del df_calendar['week']
del df_calendar['year']

### Unificar df_sales y df_calendar

In [ ]:
# Unión con el dataframe del calendario
df_sales_calendar = pd.merge(df_sales_net, df_calendar, on='d', how='left')

In [ ]:
df_sales_calendar.head()

In [ ]:
# df_sales_calendar es una tabla muy pesada, se eliminan primeramente aquellas columnas irrelevantes
del df_sales_calendar['d']            # utilizada para hacer el merge
del df_sales_calendar['event']        # columna con exceso de nulos

In [ ]:
df_sales_calendar.info()

In [ ]:
df_sales_calendar['year_week']=df_sales_calendar['year_week'].astype('float64')

### Optimización de df_prices

In [ ]:
df_prices.head()

In [ ]:
df_prices.info()

In [ ]:
# transformamos prices para optimizar la memoria
df_prices["sell_price"] = df_prices["sell_price"].astype("float32")

In [ ]:
df_prices.isnull().sum()

In [ ]:
# Se crean las mismas columnas de fecha que hay en df_sales_calendar para poder hacer el merge correctamente

df_prices['year_week']=df_prices['yearweek']
df_prices['year_week'].fillna(1990-11, inplace=True)

# Crear id, no es necesario ya que se puede hacer merge por category + item, pero será más cómodo
df_prices["id"] = df_prices["item"] + "_" + df_prices["store_code"]

df_prices.head()

In [ ]:
# Se elimanan aquellas columnas innecesarias para hacer el merge, puesto que acabarán estando repetidas o no son necesarias

df_prices.drop(["item", "category", "store_code", "yearweek"], axis=1, inplace=True)
df_prices.head()

In [ ]:
df_prices.isnull().sum()

## 3. Merge final

In [ ]:
# Finalmente, uniremos la tabla combinada de ventas y calendario con la tabla de precios
df_combined = pd.merge(df_sales_calendar, df_prices,
                          on=['id', 'year_week'],
                          how='left')

In [ ]:
df_combined.head()

In [ ]:
# creo que usaremos este dataset para power bi, ya que ahi podemos hacer graficas por año, mes, semanas...
# creando las columnas directamente en power bi
# elimino las siguientes para que pese menos
del df_combined['weekday']
del df_combined['weekday_int']


In [ ]:
df_combined.info()

In [ ]:
df_combined['year_week']=df_combined['year_week'].astype('uint16')

In [ ]:
#podemos guardar un pickle de los datos totales para hacer los cálculos en Power BI
#df_combined.to_csv('gb_union_days.csv', index=False)

## 4. GROUPBY por semana

Para poder manipular el dataset de manera óptima se deciden agrupar los datos semanalmente. Del merge del dataset de sales con calendar debemos obtener dos datasets auxiliares:

df_sales_calendar1: incluirá las columnas date, year_week y id. De este nos quedaremos con la primera fecha de la semana según year_week y id.
df_sales_calendar2: incluirá las columnas quantity, year_week y id. De este sumará las cantidades según year_week y id.

In [ ]:
df_sales_calendar.head()

In [ ]:
df_sales_calendar1=df_sales_calendar

In [ ]:
df_sales_calendar1.info()

In [ ]:
to_keep = ["id", "date", "year_week"]
to_drop = df_sales_calendar1.columns.difference(to_keep)
df_sales_calendar1 = df_sales_calendar1.drop(columns=to_drop)
df_sales_calendar1.head()

In [ ]:
date_id = df_sales_calendar1.sort_values(by=['date', "year_week", "id"]).reset_index(drop=True)
date_id.head()

In [ ]:
# Seleccionar la primera fecha para cada grupo "year_week", "id"
date_id = date_id.groupby(["year_week", "id"]).first()
date_id

In [ ]:
df_sales_calendar2=df_sales_calendar

In [ ]:
# columnas que necesito para poder hacer el merge posteriormente con el resto de tablas
keep = ["id", "quantity", "year_week"]
drop = df_sales_calendar2.columns.difference(keep)
gb_id_yearweek = df_sales_calendar2.drop(columns=drop)

In [ ]:
# agrupamos por las columnas que juntas dan una combinación única
gb_id_yearweek = gb_id_yearweek.groupby(["year_week", "id"]).sum()
gb_id_yearweek

In [ ]:
# reseteamos el índice para poder hacer merge por year_week e id
gb_id_yearweek= gb_id_yearweek.reset_index()
gb_id_yearweek.head()

In [ ]:
df_sales.head()

In [ ]:
# me quedo solo con las primeras 7 columnas
df_sales_net = df_sales.iloc[:, :7]
df_sales_net.head()

In [ ]:
df_sales_calendar_gb = pd.merge(
    left = gb_id_yearweek,
    right = df_sales_net,
    how = "left",
    on = ["id"]
)

df_sales_calendar_gb.head()

In [ ]:
df_sales_calendar_gb.shape

In [ ]:
df_prices.head()

In [ ]:
gb_union = pd.merge(
    left = df_sales_calendar_gb,
    right = df_prices,
    how = "left",
    on = ["id","year_week" ]
)

gb_union.head()

In [ ]:
date_id.head()

In [ ]:
# reseteamos el índice para poder hacer merge
date_id= date_id.reset_index()
date_id.head()

In [ ]:
gb_union_weeks = pd.merge(
    left = gb_union,
    right = date_id,
    how = "left",
    on = ["id","year_week" ]
)

gb_union_weeks.head()

In [ ]:
gb_union_weeks.shape

## 5. CSV

In [ ]:
gb_union_weeks.to_csv('gb_union_weeks.csv', index=False)